In [1]:
import pygrib
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import re
import random
from collections import defaultdict
import datetime

In [2]:
input_dir = "/N/project/Zli_lab/Data/Observations/stage4_jja_processed"
grib_files = sorted(glob.glob(os.path.join(input_dir, "ST4.*.01h")))

output_dir = "/N/project/Zli_lab/gongg/stage4_data/stage4_nc"

print(datetime.datetime.now())
for grb_file in grib_files[20000:40000]:
    base_name = os.path.basename(grb_file)  
  
    time_str = base_name.split(".")[1]   
    
    time_dt = datetime.datetime.strptime(time_str, "%Y%m%d%H")


    grbs = pygrib.open(grb_file)
    
    selected_grb = grbs.select(name="Total Precipitation")[0]
    
    data, lat, lon = selected_grb.data()
    grbs.close()

    
    array_2d = data.filled(np.nan)

    
    ds = xr.Dataset(
        {
            "tp": (["y", "x"], array_2d)
        },
        coords={
            "lat": (["y", "x"], lat),
            "lon": (["y", "x"], lon),
           
            "time": time_dt
        }
    )

    
    nc_filename = os.path.join(output_dir, f"{time_str}.nc")
    ds.to_netcdf(nc_filename)

print(datetime.datetime.now())

2025-03-20 22:14:36.375663


ValueError: no matches found

In [2]:
input_dir = "/N/project/Zli_lab/Data/Observations/stage4_jja_processed"
output_dir = "/N/project/Zli_lab/gongg/stage4_data/stage4_nc"


grib_files = sorted(glob.glob(os.path.join(input_dir, "ST4.*.01h")))
input_times = {os.path.basename(f).split(".")[1]: f for f in grib_files}


nc_files = sorted(glob.glob(os.path.join(output_dir, "*.nc")))
output_times = {os.path.splitext(os.path.basename(f))[0] for f in nc_files}


missing_times = set(input_times.keys()) - output_times

In [5]:
sorted_missing = sorted(missing_times)


for time_str in sorted_missing[5000:10000]:
    grb_file = input_times[time_str]
    try:
        time_dt = datetime.datetime.strptime(time_str, "%Y%m%d%H")
        grbs = pygrib.open(grb_file)
        selected_grb = grbs[1]  # 注意：pygrib的索引从1开始
        data, lat, lon = selected_grb.data()
        grbs.close()

        array_2d = data.filled(np.nan)

        ds = xr.Dataset(
            {
                "tp": (["y", "x"], array_2d)
            },
            coords={
                "lat": (["y", "x"], lat),
                "lon": (["y", "x"], lon),
                "time": time_dt
            }
        )

        nc_filename = os.path.join(output_dir, f"{time_str}.nc")
        ds.to_netcdf(nc_filename)
    except Exception:
        pass

### step

In [34]:
# 目录路径
data_dir = "/N/project/Zli_lab/Data/Observations/stage4_jja_processed"

# 为两种文件格式分别定义匹配模式：
# 1) ST4.*09*.01h*
# 2) st4_conus.*09*.01h*.grb2
pattern_st4  = os.path.join(data_dir, "ST4.????09????.01h")
pattern_conus = os.path.join(data_dir, "st4_conus.????09????.01h.grb2")

# 分别获取匹配的文件，并合并、排序
files_sep = glob.glob(pattern_st4) + glob.glob(pattern_conus)
files_sep_sorted = sorted(files_sep)

In [37]:
# 目录路径
data_dir = "/N/project/Zli_lab/Data/Observations/stage4_jja_processed"

# 为两种文件格式分别定义匹配模式：
# 1) ST4.*09*.01h*
# 2) st4_conus.*09*.01h*.grb2
pattern_st4  = os.path.join(data_dir, "ST4.????09????.01h")
pattern_conus = os.path.join(data_dir, "st4_conus.????09????.01h.grb2")

# 分别获取匹配的文件，并合并、排序
files_sep = glob.glob(pattern_st4) + glob.glob(pattern_conus)
files_sep_sorted = sorted(files_sep)

In [39]:
sorted_missing = files_sep_sorted

for file_path in sorted_missing[0:5000]:
    try:
        # 从文件名中解析时间 (如果需要)
        basename = os.path.basename(file_path)
        date_part = basename.split('.')[1]  # 例如 "2002090100"
        time_dt = datetime.datetime.strptime(date_part, "%Y%m%d%H")

        # 直接用 file_path 打开 GRIB 文件
        grbs = pygrib.open(file_path)
        selected_grb = grbs[1]
        data, lat, lon = selected_grb.data()
        grbs.close()

        array_2d = data.filled(np.nan)

        ds = xr.Dataset(
            {
                "tp": (["y", "x"], array_2d)
            },
            coords={
                "lat": (["y", "x"], lat),
                "lon": (["y", "x"], lon),
                "time": time_dt
            }
        )

        nc_filename = os.path.join(output_dir, f"{date_part}.nc")
        ds.to_netcdf(nc_filename)
    except Exception as e:
        print("Error:", e)
        pass
